In [114]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import colorsys

plt.rcParams['figure.figsize'] = (20, 20)


def show_pics(plot_colum_size, plot_row_size, imgs, **kargs):
    for i in range(plot_colum_size * plot_row_size):
        plt.subplot(plot_row_size, plot_colum_size, i + 1)
        img = imgs[i]
        d = len(img.shape)
#         if d != 3:
#             kargs['cmap'] = 'gray'
        plt.imshow(img, **kargs)
    plt.show()
    
def cv_show_pics(plot_colum_size, plot_row_size, imgs, **kargs):
    cv_imgs = []
    for img in imgs:
        cv_imgs.append(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    show_pics(plot_colum_size, plot_row_size, cv_imgs, **kargs)
    
def draw_box(box, frame):
    pts = cv.boxPoints(box)
    pts = np.int0(pts)
    frame = cv.polylines(frame,[pts],True, 255,2)
    a, _, b, _ = pts
    cx = int((a[0] + b[0]) / 2)
    cy = int((a[1] + b[1]) / 2)
    frame = cv.circle(frame, (cx, cy), 10, (255, 0, 0), -1)
    return frame

In [115]:
img_dir = "../images/"
# r, h, c, w = 450, 100, 890, 100  # rubber
# r,h,c,w = 250,300,850,75 # election day bracelet
# r,h,c,w = 200,200,800,350 # blue
# r, h, c, w = 200, 400, 525, 300  # head
r, h, c, w = 75, 425, 870, 290  # german
track_window = (c, r, w, h)

cap = cv.VideoCapture(img_dir + "german.mp4")
_, first_frame = cap.read()
roi = first_frame[r:r + h, c:c + w, :]
roi_hsv = cv.cvtColor(roi, cv.COLOR_BGR2HSV)
# cv_show_pics(2, 1, [roi, first_frame])
first_frame_hsv = cv.cvtColor(first_frame, cv.COLOR_BGR2HSV)
# cv_show_pics(1, 1, [first_frame_hsv])

In [116]:
# bucket_n = 256
# channels = [0] # [0, 1, 2]
# ranges = [0, 255] # [0, 255, 0, 255, 0, 255]
# buckets = [bucket_n] # [bucket_n, bucket_n, bucket_n]
bucket_n = 8
channels = [0, 1, 2]
ranges = [0, 255, 0, 255, 0, 255]
buckets = [bucket_n, bucket_n, bucket_n]
mask = None

roi_hist = cv.calcHist([roi_hsv], channels, mask, buckets, ranges)
roi_hist = cv.normalize(roi_hist, roi_hist, 0, 255, cv.NORM_MINMAX)

In [117]:
# plt.rcParams['figure.figsize'] = (40, 30)
# y = roi_hist.flatten()
# N = len(y)
# x = range(N)
# width = 1
# plt.bar(x, y, width, color=[colorsys.hsv_to_rgb(i/N, 1, 1) for i in range(N)])
# plt.axis('off')
# plt.show()

In [118]:
%%time
term_crit = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1)
fourcc = cv.VideoWriter_fourcc(*'DIVX')
h, w, _ = first_frame.shape
out = cv.VideoWriter('output.mkv', fourcc, 25.0, (2 * w, h), isColor=True)
while True:
    ret, frame = cap.read()

    if ret:
        frame_hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        prob_map = cv.calcBackProject([frame_hsv], channels, roi_hist, ranges, 1)
        # ret, track_window = cv.meanShift(prob_map, track_window, term_crit)
        ret, track_window = cv.CamShift(prob_map, track_window, term_crit)
        # x, y, w, h = track_window
        # frame = cv.rectangle(frame, (x, y), (x + w, y + h), 255, 10)
        frame = draw_box(ret, frame)
        prob_map = cv.cvtColor(prob_map, cv.COLOR_GRAY2BGR)
        prob_map = draw_box(ret, prob_map)
        frame = np.concatenate([frame, prob_map], axis=1)
        out.write(frame)
    else:
        break
out.release()

Wall time: 13.6 s


In [119]:
cap.release()